# Анализ автомобильного рынка Казахстана

В датасете представлены данные о продажах автомобилей на территории Казахстана в январе-сентябре 2019 года всеми официальными автодилерами, которые собрала статистическая компания-агрегатор.

Данные содержат ошибки и в текущем виде не пригодны для анализа. Задача — очистить данные до состояния, когда их можно анализировать

Описание данных:

Год – год продажи (2019),  
Месяц – месяц продажи (январь - сентябрь),  
Компания – название автоцентра,  
Бренд – название продаваемой марки автомобиля,  
Модель – название модели автомобиля,  
Модификация – модификация модели автомобиля (удаляем),  
Год выпуска – год производства автомобиля,  
Страна-производитель – страна, где произведен автомобиль,  
Вид топлива – бензин, дизель, электричество, гибрид,  
Объём двиг, л – объем двигателя автомобиля в литрах,  
Коробка передач – тип коробки переключения передач (оставляем два варианта: автоматическая, механическая, то есть все что не механика ставим автомат, на DSG, S-Tronic и прочее делить не надо, равно как и количество передач),  
Тип привода – в итоге оставляем RWD – задний привод, FWD – передний привод, 4WD – полный привод, 2WD – все остальное (подключаемый полный привод и где нет четкого указания передний или задний это привод)  
Сегмент – сегмент, к которому относится авто (удаляем)  
Регион – регион продажи  
Наименование дилерского центра – совпадает с компанией – можно удалить  
Тип клиента – юридическое или физическое лицо (в рамках анализа не критично – можно удалить)  
Форма расчета – наличный и безналичный расчет (много пропусков – можно удалять)  
Количество – количество автомобилей в заказе  
Цена USD – цена автомобиля  
Продажа USD – цена заказа (цена авто умноженная на количество и за вычетом скидок если есть)  
Область – область продажи  
Сегментация 2013 – сегмент автомобиля актуальный  
Класс 2013 – класс автомобиля актуальный  
Сегментация Eng – английская сегментация (удаляем)  
Локализация производства – удаляем (совпадает со страной производителем)  

Признаки-категории:  
сегмент  
класс  
тип привода  
коробка передач  

In [1]:
#импортируем бибилотеки
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
import statistics as st
import math
import warnings

In [2]:
#отключаем вывод предупреждений
warnings.filterwarnings("ignore")

In [3]:
# количество выводимых рядов
pd.set_option('display.max_rows', 1000)
# число столбцов
pd.set_option('display.max_columns', 30)
# оличество символов в записи
pd.set_option('display.max_colwidth', None)
# количество элементов в списке
pd.set_option('display.max_seq_items', None)
#количество знаков десятичных разрядов
pd.set_option('display.float_format', '{:.2f}'.format)

In [4]:
#загрузим файл с данными
import pandas as pd
auto = pd.read_csv('C:/Users/mrsag/Desktop/данные для проекта/auto_kz_2019_датасет.csv', sep=';', decimal=',')

In [5]:
#общая информация о датасете
auto.head()

,Год,Месяц,Компания,Бренд,Модель,Модификация,Год выпуска,Страна-производитель,Вид топлива,"Объём двиг, л,",Коробка передач,Тип привода,Сегмент,Регион,Наименование дилерского центра,Тип клиента,Форма расчета,Количество,"Цена, USD","Продажа, USD",Область,Сегментация 2013,Класс 2013,Сегментация Eng,Локализация производства
0,2019,Май,Mercur Auto,Audi,A3,TFSI,2018,Германия,Бензин,"1,4",S-tronic,передний,Легковые автомобили Premium,Алматы,Mercur Auto Алматы,Физ. Лицо,безналичный,1.00,28115.00,28115.00,г.Алматы,Легковые автомобили,C класс,C,Импорт
1,2019,Август,Mercur Auto,Audi,A3,TFSI,2018,Германия,Бензин,"1,4",S-tronic,передний,Легковые автомобили Premium,Алматы,Mercur Auto Алматы,Юр. Лицо,наличный,1.00,32246.99,32246.99,г.Алматы,Легковые автомобили,C класс,C,Импорт
2,2019,Апрель,Mercur Auto,Audi,A4,TFSI,2018,Германия,Бензин,"1,4",S-Tronic,FWD,Легковые автомобили Premium,Алматы,Mercur Auto Алматы,Физ. Лицо,безналичный,1.00,32000.00,32000.00,г.Алматы,Легковые автомобили,D класс,D,Импорт
3,2019,Июль,Mercur Auto,Audi,A4,TFSI,2018,Германия,Бензин,"1,4",S-tronic,передний,Легковые автомобили Premium,Алматы,Mercur Auto Алматы,Юр. Лицо,безналичный,1.00,31929.00,31929.00,г.Алматы,Легковые автомобили,D класс,D,Импорт
4,2019,Июль,Mercur Auto,Audi,A4,TFSI,2018,Германия,Бензин,"1,4",S-tronic,передний,Легковые автомобили Premium,Алматы,Mercur Auto Алматы,Физ. Лицо,наличный,1.00,31929.00,31929.00,г.Алматы,Легковые автомобили,D класс,D,Импорт


In [6]:
#исходный размер датасета
start_shape = auto.shape
print(f'Итого имеем {start_shape[0]} наблюдений с {start_shape[1]} столбцами')

Итого имеем 39966 наблюдений с 25 столбцами


In [7]:
#список колонок
old_columns = auto.columns.tolist()
print(old_columns)

['Год', 'Месяц', 'Компания', 'Бренд', 'Модель', 'Модификация', 'Год выпуска', 'Страна-производитель', 'Вид топлива', 'Объём двиг, л,', 'Коробка передач', 'Тип привода', 'Сегмент', 'Регион', 'Наименование дилерского центра', 'Тип клиента', 'Форма расчета', 'Количество', 'Цена, USD', 'Продажа, USD', 'Область', 'Сегментация 2013', 'Класс 2013', 'Сегментация Eng', 'Локализация производства']


In [8]:
#список новых колонок
new_columns = ['year', 'month', 'company', 'brand', 'model', 'modification', 'release_year', 'country', 'fuel_type',
         'engine_volume', 'transmission_type', 'drive_type', 'segment', 'region', 'dealer', 'client_type', 'payment_type',
         'quantity', 'price_usd', 'sell_price_usd', 'state', 'segment_2013', 'class_2013', 'segment_eng', 'product_localization']

In [9]:
#соответсивие двух списков
list(zip(old_columns, new_columns))
for key, value in zip(old_columns, new_columns):
    print(key, ':', value)

Год : year
Месяц : month
Компания : company
Бренд : brand
Модель : model
Модификация : modification
Год выпуска : release_year
Страна-производитель : country
Вид топлива : fuel_type
Объём двиг, л, : engine_volume
Коробка передач : transmission_type
Тип привода : drive_type
Сегмент : segment
Регион : region
Наименование дилерского центра : dealer
Тип клиента : client_type
Форма расчета : payment_type
Количество : quantity
Цена, USD : price_usd
Продажа, USD : sell_price_usd
Область : state
Сегментация 2013 : segment_2013
Класс 2013 : class_2013
Сегментация Eng : segment_eng
Локализация производства : product_localization


In [10]:
#замена названия колонок
new_col_dict = dict(zip(old_columns, new_columns))
auto = auto.rename(columns=new_col_dict)

In [11]:
#общая информация
auto.head()

,year,month,company,brand,model,modification,release_year,country,fuel_type,engine_volume,transmission_type,drive_type,segment,region,dealer,client_type,payment_type,quantity,price_usd,sell_price_usd,state,segment_2013,class_2013,segment_eng,product_localization
0,2019,Май,Mercur Auto,Audi,A3,TFSI,2018,Германия,Бензин,"1,4",S-tronic,передний,Легковые автомобили Premium,Алматы,Mercur Auto Алматы,Физ. Лицо,безналичный,1.00,28115.00,28115.00,г.Алматы,Легковые автомобили,C класс,C,Импорт
1,2019,Август,Mercur Auto,Audi,A3,TFSI,2018,Германия,Бензин,"1,4",S-tronic,передний,Легковые автомобили Premium,Алматы,Mercur Auto Алматы,Юр. Лицо,наличный,1.00,32246.99,32246.99,г.Алматы,Легковые автомобили,C класс,C,Импорт
2,2019,Апрель,Mercur Auto,Audi,A4,TFSI,2018,Германия,Бензин,"1,4",S-Tronic,FWD,Легковые автомобили Premium,Алматы,Mercur Auto Алматы,Физ. Лицо,безналичный,1.00,32000.00,32000.00,г.Алматы,Легковые автомобили,D класс,D,Импорт
3,2019,Июль,Mercur Auto,Audi,A4,TFSI,2018,Германия,Бензин,"1,4",S-tronic,передний,Легковые автомобили Premium,Алматы,Mercur Auto Алматы,Юр. Лицо,безналичный,1.00,31929.00,31929.00,г.Алматы,Легковые автомобили,D класс,D,Импорт
4,2019,Июль,Mercur Auto,Audi,A4,TFSI,2018,Германия,Бензин,"1,4",S-tronic,передний,Легковые автомобили Premium,Алматы,Mercur Auto Алматы,Физ. Лицо,наличный,1.00,31929.00,31929.00,г.Алматы,Легковые автомобили,D класс,D,Импорт


In [12]:
auto.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39966 entries, 0 to 39965
Data columns (total 25 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   year                  39966 non-null  int64  
 1   month                 39966 non-null  object 
 2   company               39966 non-null  object 
 3   brand                 39966 non-null  object 
 4   model                 39966 non-null  object 
 5   modification          36375 non-null  object 
 6   release_year          39465 non-null  object 
 7   country               39966 non-null  object 
 8   fuel_type             36826 non-null  object 
 9   engine_volume         35708 non-null  object 
 10  transmission_type     36711 non-null  object 
 11  drive_type            35677 non-null  object 
 12  segment               33205 non-null  object 
 13  region                39966 non-null  object 
 14  dealer                39966 non-null  object 
 15  client_type        

In [13]:
#замена типа данных
auto['engine_volume'] = auto['engine_volume'].astype('str')

In [14]:
#количество пропусков
auto.isna().sum().sort_values()

year                        0
class_2013                  0
segment_2013                0
state                       0
sell_price_usd              0
price_usd                   0
dealer                      0
region                      0
segment_eng                 0
product_localization        0
country                     0
model                       0
brand                       0
company                     0
month                       0
engine_volume               0
quantity                    6
release_year              501
fuel_type                3140
transmission_type        3255
modification             3591
drive_type               4289
segment                  6761
client_type              7047
payment_type            25928
dtype: int64

In [15]:
#процент пропусков
display('% пропусков:',round(auto.isna().sum()/len(auto)*100,2).sort_values())

'% пропусков:'

year                    0.00
class_2013              0.00
segment_2013            0.00
state                   0.00
sell_price_usd          0.00
price_usd               0.00
dealer                  0.00
region                  0.00
segment_eng             0.00
product_localization    0.00
country                 0.00
model                   0.00
brand                   0.00
company                 0.00
month                   0.00
engine_volume           0.00
quantity                0.02
release_year            1.25
fuel_type               7.86
transmission_type       8.14
modification            8.99
drive_type             10.73
segment                16.92
client_type            17.63
payment_type           64.88
dtype: float64

In [16]:
#информация о мин, макс, средних значениях
auto.describe()

,year,quantity,price_usd,sell_price_usd
count,39966.00,39960.00,39966.00,39966.00
mean,2019.00,1.35,28250.16,33757.67
std,0.00,2.37,20005.76,50702.45
min,2019.00,-1.00,7100.00,-35588.25
25%,2019.00,1.00,15518.50,16601.93
50%,2019.00,1.00,24282.27,26248.96
75%,2019.00,1.00,32505.66,35000.00
max,2019.00,115.00,328510.97,3778206.26


Проверка дубликатов

In [17]:
#дубликаты
auto.duplicated().sum()

18698

In [18]:
#приведем названия к нижнему регистру
auto['company'] = auto['company'].str.lower()

In [19]:
#уникальные названия команий
auto['company'].unique()

array(['mercur auto', 'автоцентр-бавария', 'бипэк авто', 'вираж',
       'eurasia motor premium', 'daewoo bus kazakhstan', 'caspian motors',
       'каспиан моторс', 'семаз', 'astana motors', 'astana motors ',
       'hyundai com trans kazakhstan ', 'nissan manufacturing rus',
       'свс-транс', 'allur auto', 'almaty motors premium', 'terra motors',
       'тк камаз', 'toyota motor kazakhstan', 'лифан моторс рус',
       'man truck & bus kazakhstan', 'автомир гк', 'autokapital',
       'автокапитал', 'ммс рус', 'mmc rus', 'mercur autos',
       'узавто-казахстан', 'равон моторс казахстан',
       'ravon motors kazakstan', 'renault россия', 'scania central asia',
       'subaru kazakhstan', 'volkswagen group rus', 'автодом motors kst',
       'автомир-центр', 'тоо "eurasia motor zhaik"',
       'scandinavian motors', 'хино моторс казахстан', 'hino motors '],
      dtype=object)

In [20]:
#уберем неявные дубликаты
auto = auto.apply(lambda x:x.replace(
    {'mercur autos':'mercur auto','каспиан моторс':'caspian motors', 'ммс рус':'mmc rus','автокапитал':'autokapital',
     'равон моторс казахстан':'ravon motors kazakstan', 'astana motors ':'astana motors',
     'хино моторс казахстан':'hino motors','тоо "eurasia motor zhaik"':'тоо eurasia motor zhaik', 'hyundai com trans kazakhstan ': 'hyundai com trans kazakhstan'}, regex=True))      

In [21]:
auto['company'].unique()

array(['mercur auto', 'автоцентр-бавария', 'бипэк авто', 'вираж',
       'eurasia motor premium', 'daewoo bus kazakhstan', 'caspian motors',
       'семаз', 'astana motors', 'hyundai com trans kazakhstan',
       'nissan manufacturing rus', 'свс-транс', 'allur auto',
       'almaty motors premium', 'terra motors', 'тк камаз',
       'toyota motor kazakhstan', 'лифан моторс рус',
       'man truck & bus kazakhstan', 'автомир гк', 'autokapital',
       'mmc rus', 'узавто-казахстан', 'ravon motors kazakstan',
       'renault россия', 'scania central asia', 'subaru kazakhstan',
       'volkswagen group rus', 'автодом motors kst', 'автомир-центр',
       'тоо eurasia motor zhaik', 'scandinavian motors', 'hino motors',
       'hino motors '], dtype=object)

In [22]:
#проверим колонку "год выпуска"
auto['release_year'].unique()

array(['2018', '2017', '2019', nan, '2016', '2014', '2013', '2011',
       '2\xa0018', '2\xa0019'], dtype=object)

In [23]:
# заменим ошибки
error_years = {'2\xa0018':'2018', '2\xa0019': '2019'}
auto['release_year'] = auto['release_year'].replace(error_years, regex=True)

In [24]:
auto['release_year'].unique()

array(['2018', '2017', '2019', nan, '2016', '2014', '2013', '2011'],
      dtype=object)

In [25]:
#уникальные названия стран
auto['country'].unique()

array(['Германия', 'США', 'Австрия', 'Республика Казахстан',
       'Российская Федерация', 'Корея', 'Япония', 'Таиланд', 'Китай',
       'UK', 'Узбекистан', 'Венгрия', 'Турция', 'Испания', 'Нидерланды',
       'Польша', 'Швеция', 'Белоруссия', 'Бельгия'], dtype=object)

In [26]:
countries_codes = (pd.read_html('https://www.artlebedev.ru/country-list/', encoding='utf8')[0]
 [['Наименование', 'Полное наименование', 'Alpha3']]
 .rename(columns={'Наименование':'name', 'Полное наименование':'full_name'}))

In [27]:
countries_codes_dict = {}
for i in range(len(countries_codes)):
    if countries_codes.loc[i]['name'] not in countries_codes_dict:
        countries_codes_dict[countries_codes.loc[i]['name']] = countries_codes.loc[i]['Alpha3']
    if countries_codes.loc[i]['full_name'] not in countries_codes_dict:
        countries_codes_dict[countries_codes.loc[i]['full_name']] =countries_codes.loc[i]['Alpha3']

In [28]:
auto.country = auto.country.apply(lambda cell: countries_codes_dict.get(cell, cell))

In [29]:
auto.country.unique()

array(['DEU', 'США', 'AUT', 'KAZ', 'RUS', 'Корея', 'JPN', 'THA', 'CHN',
       'UK', 'UZB', 'HUN', 'TUR', 'ESP', 'NLD', 'POL', 'SWE',
       'Белоруссия', 'BEL'], dtype=object)

In [30]:
auto.country = (
    auto.country.str.replace('США', 'USA')
    .str.replace('Корея', 'KOR')
    .str.replace('Белоруссия', 'BLR'))
auto.country.unique()

array(['DEU', 'USA', 'AUT', 'KAZ', 'RUS', 'KOR', 'JPN', 'THA', 'CHN',
       'UK', 'UZB', 'HUN', 'TUR', 'ESP', 'NLD', 'POL', 'SWE', 'BLR',
       'BEL'], dtype=object)

In [31]:
#уникальные значения вида топлива
auto['fuel_type'].unique()

array(['Бензин', 'Дизель', nan, 'дизель ', 'бензин ', 'гибрид', '2',
       '1,6', '0', 'Электро', 'Электричество'], dtype=object)

In [32]:
auto['fuel_type'] = auto['fuel_type'].str.lower()

In [33]:
#удаляем лишние пропуски
auto['fuel_type'].replace(regex=True,inplace=True,to_replace=r'\s{1,}',value=r'')
auto['fuel_type'].unique()

array(['бензин', 'дизель', nan, 'гибрид', '2', '1,6', '0', 'электро',
       'электричество'], dtype=object)

In [34]:
#создадим функцию categorize_drive_type для распределения типов привода на категории
def categorize_fuel_type(cell):
  if cell in ['бензин', '2', '1,6', '0']:
    return 'F' #бензин

  elif cell in ['дизель']:
    return 'D'#дизель

  elif cell in ['электро','электричество']:
    return 'E' #электро

  return 'HYB' #гибрид

In [35]:
auto['fuel_type'] = auto['fuel_type'].apply(categorize_fuel_type)
auto['fuel_type'].unique()

array(['F', 'D', 'HYB', 'E'], dtype=object)

In [36]:
#уникальные значения объема двигателя
auto['engine_volume'].unique()

array(['1,4', '2', '3', '4,4', '6,6', '1,5', '1,596', '2,4', '1,7',
       '26,7', '25,7', '24,7', '23,7', '22,7', '21,7', '20,7', '19,7',
       '18,7', '17,7', '16,7', '15,7', '14,7', '13,7', '12,7', '11,7',
       '10,7', '9,7', '8,7', '7,7', '6,7', '5,7', '4,7', '3,7', '2,7',
       '6,2', '1,4 Turbo', '2,5', '1,6', 'nan', '2,8', '4,3', '4,3,',
       '1,6 MPI', '2,0 MPI', '2,4 GDI', '2,4 MPI', '2,5 CRDI VGT',
       '2,5 CRDI WGT', '3,9', '7,5', '12,3', '1,6 T-GDI', '2,0 CRDI',
       '2.0', '3,5', '5,6', '5,2', '3,0 L', '1.6', '1.5', '5', '1,598',
       '1,248', '1,998', '2,359', '1,999', '3,342', '1,591', '3,47',
       '1,69', '1,774', '2.5', '2.7 ', '3.5', '2.7', '4.6', '4,6', '5.7',
       '1,8', '10,5', '4', '5,5', '12', '12,8', '11', '2,2', '1,2',
       '1,485', 'MT', 'AT', '0', '13', '1,33', '#Н/Д', '4.0', '2.4G',
       '2,4G', '2.8', '2,693', '2,0', '3,8', '3.8', '1,2T', '3,6', '7,6',
       '4,9', '2.0h', '2,9', '400 Л.С.', '4,98 L,', '4,98', '88 KWH',
       '8,4 L,'

In [37]:
#корректиурем данные
auto.loc[auto.engine_volume == '400 Л.С.', 'engine_volume'] = 2
auto.loc[auto.engine_volume == '88 KWH', 'engine_volume'] = 0
auto = auto.query('engine_volume != ["MT","AT","#Н/Д"]')


In [38]:
#заменяем значения
auto['engine_volume'] = auto['engine_volume'].astype('str')
auto['engine_volume'] = auto['engine_volume'].replace({',':'.', 'Turbo':'','MPI':'','GDI':'','CRDIVGT':'',
                                      '1.6 T-GDI':'1.6','CRDI':'','WGT':'','KWH':'','400 Л.С.':'2',
                                      'VGT':'','1.6 T':'1.6', '1.6-':'1.6','3.0 L':'3.0','1.2T':'1.2','2.0h':'2.0',
                                      '4.98 L.':'4.98','8.4 L.':'8.4','6.7L':'6.7','2.4G':'2.4'},regex=True)
auto['engine_volume'] = \
[x.replace(',', '.') for x in auto['engine_volume']]
auto['engine_volume'].unique()    

array(['1.4', '2', '3', '4.4', '6.6', '1.5', '1.596', '2.4', '1.7',
       '26.7', '25.7', '24.7', '23.7', '22.7', '21.7', '20.7', '19.7',
       '18.7', '17.7', '16.7', '15.7', '14.7', '13.7', '12.7', '11.7',
       '10.7', '9.7', '8.7', '7.7', '6.7', '5.7', '4.7', '3.7', '2.7',
       '6.2', '1.4 ', '2.5', '1.6', 'nan', '2.8', '4.3', '4.3.', '1.6 ',
       '2.0 ', '2.4 ', '2.5  ', '3.9', '7.5', '12.3', '1.6-', '2.0',
       '3.5', '5.6', '5.2', '3.0', '5', '1.598', '1.248', '1.998',
       '2.359', '1.999', '3.342', '1.591', '3.47', '1.69', '1.774',
       '2.7 ', '4.6', '1.8', '10.5', '4', '5.5', '12', '12.8', '11',
       '2.2', '1.2', '1.485', '0', '13', '1.33', '4.0', '2.693', '3.8',
       '3.6', '7.6', '4.9', '2.9', '4.98', '8.4', '6.5'], dtype=object)

In [39]:
#уникальные значения типа коробки передач
auto['transmission_type'].unique()

array(['S-tronic', 'S-Tronic', 'Tiptronic', 'АКПП', 'АКП', 'Steptronic',
       'AT', '6 АТ', '5 МТ', '6АТ', 'MT', '8AT', '5МТ', 'Powershift S6',
       '6AT', nan, 'Мех.', '6 AT', '6 MT', '6MT', '5 AT', '5AT', 'МКПП',
       '7 DCT', '7DCT', '8 AT', 'МТ', 'CVT', 'DCT', '4 АТ', '6 МТ', '6МТ',
       '4АТ', '4AT', '8АТ', '8 АТ', '5MT', 'AMT', 'CVT (вариатор)',
       'CVT (вариATор)', 'CVT(вAриATор)', '7G-TRONIC', '9G-TRONIC',
       '12AT', '12АТ', '5АТ', '7АТ', '7AT', '6M/T', 'PDK', '8-ступ АКПП',
       '8', '8АКПП', '8 АКПП Tiptronic S', '7 АКПП (PDK)', '7 АКПП PDK',
       '8 АКПП (PDK)', 'АТ', 'АT', 'МT', '0', '7 DSG', '7DSG', '6 DSG',
       '6DSG', 'DSG', '6A', '6А', '5M', 'A/T', 'M/T', 'МКП', '5М', 'М/T',
       ' 7 АКП', '7АКП', 'А/T', '8A', 'TDI', '9AT', 'РЕДУКТОР'],
      dtype=object)

In [40]:
#удалим из названия лишние пробелы
auto['transmission_type'].replace(regex=True,inplace=True,to_replace=r'\s{1,}',value=r'')
auto['transmission_type'].unique()

array(['S-tronic', 'S-Tronic', 'Tiptronic', 'АКПП', 'АКП', 'Steptronic',
       'AT', '6АТ', '5МТ', 'MT', '8AT', 'PowershiftS6', '6AT', nan,
       'Мех.', '6MT', '5AT', 'МКПП', '7DCT', 'МТ', 'CVT', 'DCT', '4АТ',
       '6МТ', '4AT', '8АТ', '5MT', 'AMT', 'CVT(вариатор)',
       'CVT(вариATор)', 'CVT(вAриATор)', '7G-TRONIC', '9G-TRONIC', '12AT',
       '12АТ', '5АТ', '7АТ', '7AT', '6M/T', 'PDK', '8-ступАКПП', '8',
       '8АКПП', '8АКППTiptronicS', '7АКПП(PDK)', '7АКППPDK', '8АКПП(PDK)',
       'АТ', 'АT', 'МT', '0', '7DSG', '6DSG', 'DSG', '6A', '6А', '5M',
       'A/T', 'M/T', 'МКП', '5М', 'М/T', '7АКП', 'А/T', '8A', 'TDI',
       '9AT', 'РЕДУКТОР'], dtype=object)

In [41]:
#категоризация
def categorize_transmission(cell):
  if cell in ['5МТ', 'MT', '5МТ.', 'Мех', '6MT', '6МТ', '5MT', '6M/T', 'МT',
              '5M', 'M/T','МКП', '5М', 'М/T']:
    return 'MT'

  return 'AT'

In [63]:
#проверим распределение на категории
auto['transmission_type'] = auto['transmission_type'].apply(categorize_transmission)
df['transmission_type'] = df['transmission_type'].astype('category')
auto['transmission_type'].unique()

array(['AT', 'MT'], dtype=object)

In [43]:
#уникальные значения типа привода
auto['drive_type'].unique()

array(['передний', 'FWD', 'quattro', 'Задний', 'Полный', 'AWD',
       'Передний', '4WD', '4х2.2', nan, 'RWD', '2 WD', '2WD', '4 WD',
       '4X4', '4х4', '4x4', '4x2', '4х2', '4X2', 'Астана',
       'Передний (FF)', '0', 'FF', 'полный', '4Motion'], dtype=object)

In [44]:
def categorize_drive_type(cell):
  if cell in ['передний', 'FWD', 'Передний', 'Передний (FF)','FF']:
    return 'FWD' #передний привод

  elif cell in ['4WD','quattro', 'Полный', 'полный', 'AWD', '4 WD', '4Motion']:
    return '4WD'#полный привод

  elif cell in ['Задний','RWD']:
    return 'RWD' #задний привод

  return '2WD' #другие

In [45]:
#проверим распределение на категории
auto['drive_type_category'] = auto['drive_type'].apply(categorize_drive_type)
auto['drive_type_category'].unique()

array(['FWD', '4WD', 'RWD', '2WD'], dtype=object)

In [48]:
auto.drop(auto.columns[[5, 12, 14, 15, 16, 20, 23, 24]], axis= 1 , inplace= True )

In [53]:
#изменение типа клиента
auto = auto.dropna(subset = 'quantity')
auto['quantity'] = auto['quantity'].astype('int')

In [54]:
auto.query('quantity <= 0')

,year,month,company,brand,model,release_year,country,fuel_type,engine_volume,transmission_type,drive_type,region,quantity,price_usd,sell_price_usd,segment_2013,class_2013,transmission_category,drive_type_category
25728,2019,Август,бипэк авто,Skoda,Superb,2019,KAZ,F,2,6DSG,Полный,Тараз,-1,35588.25,-35588.25,Легковые автомобили,D класс,AT,4WD


In [58]:
auto.loc[auto['quantity']>=2, 'client_type'] = 'юр. лицо'
auto.loc[auto['quantity']<2, 'client_type'] = 'физ. лицо'

In [59]:
auto['client_type'].unique()

array(['физ. лицо', 'юр. лицо'], dtype=object)